**Using XGBoost to work with dataset that includes missing values**

In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

In [19]:
df = pd.read_csv('Master_Clean_w_NaN.csv', index_col=0)

In [20]:
df.head()

,SEQN,Age,Heart_Rate,Irregular_Pulse,Systolic_BP_Avg,Diastolic_BP_Avg,Weight_kg,Height_cm,BMI,Waist_cm,...,Family_Income_15.0,Family_Income_nan,Gender_1.0,Gender_2.0,Race_1.0,Race_2.0,Race_3.0,Race_4.0,Race_6.0,Race_7.0
0,83732.0,62.0,76.0,0.0,122.666667,65.333333,94.8,184.5,27.8,101.1,...,0,0,1,0,0,0,1,0,0,0
1,83733.0,53.0,72.0,0.0,140.000000,86.000000,90.4,171.4,30.8,107.9,...,0,0,1,0,0,0,1,0,0,0
2,83735.0,56.0,78.0,0.0,134.000000,70.000000,109.8,160.9,42.4,110.1,...,0,0,0,1,0,0,1,0,0,0
3,83736.0,42.0,76.0,0.0,104.000000,60.000000,55.2,164.9,20.3,80.4,...,0,0,0,1,0,0,0,1,0,0
4,83741.0,22.0,66.0,0.0,111.333333,72.666667,76.6,165.4,28.0,86.6,...,0,0,1,0,0,0,0,1,0,0


In [21]:
df.shape

(4707, 68)

In [22]:
df = df[np.isfinite(df['Target'])]

In [23]:
df.shape

(4423, 68)

In [24]:
df_target = df['Target']

In [25]:
df_target.shape

(4423,)

In [26]:
df_features = df.drop(columns=['SEQN', 'Target'])

In [27]:
df_features.shape

(4423, 66)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.2, random_state=123)

In [29]:
sum(y_train==0)

3473

In [30]:
sum(y_train==1)

65

In [31]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [32]:
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 98.73%
Validation accuracy: 98.31%


In [34]:
print('Accuracy Score:  ' + str(accuracy_score(y_test, val_preds)))
print('F1 Score:        ' + str(f1_score(y_test, val_preds)))
print('Precision Score: ' + str(precision_score(y_test, val_preds)))
print('Recall Score:    ' + str(recall_score(y_test, val_preds)) )
report = classification_report(y_test, val_preds)
print ('')
print (report)

Accuracy Score:  0.9830508474576272
F1 Score:        0.0
Precision Score: 0.0
Recall Score:    0.0

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       870
         1.0       0.00      0.00      0.00        15

   micro avg       0.98      0.98      0.98       885
   macro avg       0.49      0.50      0.50       885
weighted avg       0.97      0.98      0.97       885



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/class

In [17]:
param_grid = {
    'learning_rate': [0.1],
    'max_depth': [6],
    'min_child_weight': [10],
    'subsample': [ 0.7],
    'n_estimators': [5, 30, 100, 250],
}

In [18]:
clf = xgb.XGBClassifier(scale_pos_weight=53)
grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
val_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Grid Search found the following optimal parameters: 
learning_rate: 0.1
max_depth: 6
min_child_weight: 10
n_estimators: 250
subsample: 0.7

Training Accuracy: 99.89%
Validation accuracy: 97.29%


In [45]:
grid_clf.cv_results_['mean_test_score']

array([0.9392312 , 0.96212549, 0.96919163, 0.97427925])

In [51]:
param_grid = {
    'max_depth' : [3, 5, 6, 7],
    'learning_rate' : [0.1],
    'n_estimators' : [5, 30, 80, 100],
    'silent' : [True],
    'objective' : ['binary:logistic'],
    'booster' : ['gbtree'],
    'n_jobs' : [1],
    'nthread' : [None],
    'gamma' : [0],
    'min_child_weight' : [1, 5, 10],
    'max_delta_step' : [0],
    'subsample' : [1, 0.7],
    'colsample_bytree' : [1],
    'colsample_bylevel' : [1],
    'reg_alpha' : [0],
    'reg_lambda' : [1],
    'scale_pos_weight' : [53],
    'base_score' : [0.5],
    'random_state' : [0],
    'seed' : [None],
    'missing' : [None]
}

In [52]:
clf = xgb.XGBClassifier()
grid_clf = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
val_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Grid Search found the following optimal parameters: 
base_score: 0.5
booster: 'gbtree'
colsample_bylevel: 1
colsample_bytree: 1
gamma: 0
learning_rate: 0.1
max_delta_step: 0
max_depth: 3
min_child_weight: 10
missing: None
n_estimators: 80
n_jobs: 1
nthread: None
objective: 'binary:logistic'
random_state: 0
reg_alpha: 0
reg_lambda: 1
scale_pos_weight: 53
seed: None
silent: True
subsample: 0.7

Training Accuracy: 95.03%
Validation accuracy: 93.11%


In [56]:
best_model = grid_clf.best_estimator_

Accuracy Score:  0.9310734463276836
F1 Score:        0.18666666666666668
Precision Score: 0.11666666666666667
Recall Score:    0.4666666666666667

              precision    recall  f1-score   support

         0.0       0.99      0.94      0.96       870
         1.0       0.12      0.47      0.19        15

   micro avg       0.93      0.93      0.93       885
   macro avg       0.55      0.70      0.58       885
weighted avg       0.98      0.93      0.95       885



In [54]:
def evaluate_model(predictions, probs, train_predictions, train_probs):
    """Compare machine learning model to baseline performance.
    Computes statistics and shows ROC curve."""
    
    baseline = {}
    
    baseline['recall'] = recall_score(y_test, [0 for _ in range(len(y_test))])
    baseline['precision'] = precision_score(y_test, [0 for _ in range(len(y_test))])
    baseline['roc'] = 0.5
    
    results = {}
    
    results['recall'] = recall_score(y_test, predictions)
    results['precision'] = precision_score(y_test, predictions)
    results['roc'] = roc_auc_score(y_test, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(y_train, train_predictions)
    train_results['precision'] = precision_score(y_train, train_predictions)
    train_results['roc'] = roc_auc_score(y_train, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)} Train: {round(train_results[metric], 2)}')
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(y_test, [0 for _ in range(len(y_test))])
    model_fpr, model_tpr, _ = roc_curve(y_test, probs)

    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curves');

In [57]:
train_xgb_predictions = best_model.predict(X_train)
train_xgb_probs = best_model.predict_proba(X_train)[:,1]

xgb_predictions = best_model.predict(X_test)
xgb_probs = best_model.predict_proba(X_test)[:,1]